In [0]:
%run "../../utils/auth"

In [0]:
# Configurar display para melhor visualização
from pyspark.sql.functions import *
from pyspark.sql.types import *

print(" Notebook Bronze - Ingestão iniciada")

In [0]:
from datetime import datetime

dimension_files = ["dim_data", "dim_clinica", "dim_medico", "dim_diagnostico", "dim_exame", "dim_paciente"]
processed_date = datetime.now().strftime("%Y-%m-%d")
failed_files = []
processed_files = []

for table_name in dimension_files:
    try:
        print(f"\n Processando {table_name}...")

        # Ler CSV do landing
        df = spark.read.format("csv") \
            .option("header", "true") \
            .option("inferSchema", "true") \
            .load(f"abfss://landing@mystoacc.dfs.core.windows.net/{table_name}.csv")
        
        # Escrever em Delta na pasta Bronze
        df.write.format("delta") \
            .mode("append") \
            .save(f"abfss://bronze@mystoacc.dfs.core.windows.net/{table_name}")

        print(f"tabela: {table_name} salva na bronze")

        source = f"abfss://landing@mystoacc.dfs.core.windows.net/{table_name}.csv"
        dest = f"abfss://landing@mystoacc.dfs.core.windows.net/processed/{table_name}/{processed_date}/{table_name}.csv"
        dbutils.fs.cp(source, dest)
        dbutils.fs.rm(source)

        print(f"arquivo: {table_name} movido para processed")
        processed_files.append(table_name)

    except Exception as e:
        print(f"ERRO: {str(e)}")
        failed_files.append(table_name)

        source = f"abfss://landing@mystoacc.dfs.core.windows.net/{table_name}.csv"
        dest = f"abfss://landing@mystoacc.dfs.core.windows.net/failed/{processed_date}/{table_name}.csv"
        try:
            dbutils.fs.cp(source, dest)
            dbutils.fs.rm(source)
            print(f"Arquivo: {table_name} Movido para failed")
        except:
            print(f"Não foi possível mover arquivo")

print(f"arquivos processados: {processed_files}")
print(f"arquivos com falha: {failed_files}")


    

In [0]:
# Processar Dimensões
dimension_files = ["dim_data", "dim_clinica", "dim_medico", "dim_diagnostico", "dim_exame", "dim_paciente"]

for table_name in dimension_files:
    print(f"\n📥 Processando {table_name}...")
    
    # Ler CSV
    df = spark.read.format("csv") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .load(f"abfss://landing@mystoacc.dfs.core.windows.net/{table_name}.csv")
    
    print(f"   ✅ Lido: {df.count()} linhas")
    
    # Salvar SOMENTE como Delta (sem registrar no catálogo)
    df.write.format("delta") \
        .mode("overwrite") \
        .save(f"abfss://bronze@mystoacc.dfs.core.windows.net/{table_name}")
    
    print(f"   💾 Salvo: bronze/{table_name}")

print("\n🎉 Bronze completo!")

